# 🧩 Simple StateGraph Workflow — Concepts Explained

This code creates a **state-based workflow** using LangGraph. Instead of calling functions directly in sequence, we define:

- A **State**: a typed dictionary (`TypedDict`) that describes what data flows between nodes (`graph_state` in this case).
- **Nodes**: plain Python functions (`node_1`, `node_2`, `node_3`) that:
  - Receive the current `state` (a dict)
  - Perform some operation (here, just string concatenation)
  - Return an updated `state` (partial dict with changed values)

- A **StateGraph**: an object that stores the nodes and how they connect.
  - We add each node to the graph by name.
  - We define **edges** between nodes — this describes the execution path (`START → node_1 → node_2 → node_3 → END`).

- **Compilation**: `builder.compile()` turns the defined graph into an executable workflow.
- **Visualization**: `graph.get_graph().draw_mermaid_png()` renders a diagram of the graph so you can see the structure visually in the notebook.

**Conceptually:**  
This pattern abstracts workflows into **directed graphs**. Each node is independent, the graph controls the order, and the shared `state` moves through the graph — a foundation for building more complex, branching, or parallel agent systems.


In [0]:
from dotenv import load_dotenv
import os
import random
from typing import Literal, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph.message import add_messages
from typing_extensions import Annotated
from databricks_langchain import ChatDatabricks
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import ToolNode, tools_condition
from IPython.display import Image, display
from langgraph.graph import StateGraph, START, END


from typing_extensions import TypedDict

class State(TypedDict):
    graph_state: str

def node_1(state):
    print("---Node 1---")
    return {"graph_state": state['graph_state'] +" I am"}

def node_2(state):
    print("---Node 2---")
    return {"graph_state": state['graph_state'] +" happy!"}

def node_3(state):
    print("---Node 3---")
    return {"graph_state": state['graph_state'] +" sad!"}




# Build graph
builder = StateGraph(State)
builder.add_node("node_1", node_1)
builder.add_node("node_2", node_2)
builder.add_node("node_3", node_3)

# Logic
builder.add_edge(START, "node_1")
builder.add_edge("node_1", "node_2")
builder.add_edge("node_2", "node_3")
builder.add_edge("node_3", END)

# Add
graph = builder.compile()

# View
display(Image(graph.get_graph().draw_mermaid_png()))



In [0]:
graph.invoke({"graph_state" : "Hi, this is Anirvan."})


# 🔀 Conditional Branching in a LangGraph Workflow

In this example, we introduce **dynamic decision-making** into our StateGraph. Instead of always following a fixed sequence, a node can decide which next node to run at runtime.

We do this in three steps:

1. **Decision Function**  
   We create a Python function (here called `decide_mood`) that takes the current `state` and returns the name of the next node.  
   In this demo, it just flips a coin: 50% chance to go to `node_2`, 50% to `node_3`. In a real project, this function could look at values in `state` (like user input, an LLM output, or an API response) to pick the right branch.

2. **Attach Conditional Edges**  
   Instead of a normal `add_edge`, we use `add_conditional_edges("node_1", decide_mood)`.  
   This tells LangGraph: after running `node_1`, call `decide_mood(state)` and follow whatever node name it returns. This is how branching logic is added without hardcoding paths.

3. **Complete the Graph**  
   Both possible branches (`node_2` and `node_3`) are then connected to `END`. This ensures no matter which branch is chosen, the workflow eventually terminates cleanly.

**Conceptually:**  
- Nodes still work the same way — they process and update the `state`.  
- The graph itself becomes more flexible, able to handle different paths dynamically.  
- This pattern is the foundation for routers, conditional processing, or any workflow that depends on runtime data to decide the next step.



In [0]:
import random
from typing import Literal

def decide_mood(state) -> Literal["node_2", "node_3"]:
    
    # Often, we will use state to decide on the next node to visit
    user_input = state['graph_state'] 
    
    # Here, let's just do a 50 / 50 split between nodes 2, 3
    if random.random() < 0.5:

        # 50% of the time, we return Node 2
        return "node_2"
    
    # 50% of the time, we return Node 3
    return "node_3"

In [0]:
# Build graph
builder1 = StateGraph(State)
builder1.add_node("node_1", node_1)
builder1.add_node("node_2", node_2)
builder1.add_node("node_3", node_3)

# Logic
builder1.add_edge(START, "node_1")
builder1.add_conditional_edges("node_1", decide_mood)
builder1.add_edge("node_2", END)
builder1.add_edge("node_3", END)

# Add
graph1 = builder1.compile()

# View
display(Image(graph1.get_graph().draw_mermaid_png()))




In [0]:
graph1.invoke({"graph_state" : "Hi, this is Anirvan."})
